In [1]:
%reload_ext autoreload
%autoreload all

%aimport -torch
%aimport -matplotlib
%aimport -seaborn
%aimport -numpy
%aimport -pandas
%aimport -scipy
%aimport -lightning 

In [2]:
from eeg_snn_encoder.config import PROCESSED_DATA_DIR
from eeg_snn_encoder.dataset import CHBMITDataset, CHBMITDataModule

# Load the dataset
dataset = CHBMITDataset(PROCESSED_DATA_DIR / "stft_normalized.h5")

2025-05-08 22:18:53.942 | INFO     | eeg_snn_encoder.config:<module>:11 - PROJ_ROOT path is: /home/jupyter-group55/snn-encoder-test


In [3]:
datamodule = CHBMITDataModule(dataset, batch_size=726, worker=0)

In [6]:
bsa_studies = [
    {"name": "Classifier_BSA_Tuning", "score_limit": lambda x: x.value < 10},
    {"name": "model-tuning-bsa", "score_limit": lambda x: x.value < 12},
    {"name": "model-tuning-bsa-mse", "score_limit": lambda x: x.value < 0.26},
]

In [7]:
import optuna
import os

test_trials = []

for i in bsa_studies:
    print(f"Loading good trial from {i["name"]}")
    old_study = optuna.load_study(
        study_name=i["name"],
        storage=os.environ["OPTUNA_CONN_STRING"],
    )

    complete_trial = old_study.get_trials(
        False, states=(optuna.trial.TrialState.COMPLETE,)
    )
    filtered_trials = list(filter(i["score_limit"], complete_trial))
    sorted_trials = sorted(filtered_trials, key=lambda t: t.value)

    test_trials += sorted_trials
    print(len(sorted_trials))

Loading good trial from Classifier_BSA_Tuning
16
Loading good trial from model-tuning-bsa
5
Loading good trial from model-tuning-bsa-mse
5


In [8]:
study = optuna.create_study(
    direction="minimize",
    study_name="model-fine-tuning-bsa",
    storage=os.environ["OPTUNA_CONN_STRING"],
    load_if_exists=True,
    pruner=optuna.pruners.NopPruner()
)

[I 2025-05-08 22:19:53,434] A new study created in RDB with name: model-fine-tuning-bsa


In [10]:
from loguru import logger

import lightning.pytorch as pl
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from optuna.integration import PyTorchLightningPruningCallback

from eeg_snn_encoder.encoders import BSAEncoder
from eeg_snn_encoder.models.classifier import ModelConfig
from eeg_snn_encoder.models.lightning import LitSeizureClassifier, OptimizerConfig
import torch

torch.set_float32_matmul_precision('medium')


def objective(trial: optuna.Trial) -> float:
    model_params: ModelConfig = {
        "threshold": trial.suggest_float("threshold", 0.01, 0.5),
        "slope": trial.suggest_float("slope", 1.0, 20.0),
        "beta": trial.suggest_float("beta", 0.1, 0.99),
        "dropout_rate1": trial.suggest_float("dropout_rate1", 0.1, 0.99),
        "dropout_rate2": trial.suggest_float("dropout_rate2", 0.1, 0.99),
    }

    optimizer_params: OptimizerConfig = {
        "lr": trial.suggest_float("lr", 1e-6, 1e-4, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 1e-6, 1e-4, log=True),
        "scheduler_factor": trial.suggest_float("scheduler_factor", 0.1, 0.99),
        "scheduler_patience": trial.suggest_int("scheduler_patience", 1, 10),
    }

    encoder_params = {
        "win_size": trial.suggest_int("win_size", 1, 16),
        "cutoff": trial.suggest_float("cutoff", 0.01, 1),
        "threshold": trial.suggest_float("encoder_threshold", 0.01, 1),
    }

    spike_encoder = BSAEncoder(**encoder_params)

    lit_model = LitSeizureClassifier(
        model_config=model_params,
        optimizer_config=optimizer_params,
        spike_encoder=spike_encoder,
    )

    trainer = pl.Trainer(
        max_epochs=50,
        accelerator="auto",
        devices="auto",
        strategy="auto",
        enable_model_summary=False,
        callbacks=[PyTorchLightningPruningCallback(trial, monitor="val_mse"), EarlyStopping(monitor="val_loss", mode="min", patience=10)],
        logger=False,
    )

    trainer.fit(lit_model, datamodule=datamodule)

    return trainer.callback_metrics["val_mse"]

In [11]:
for trial in test_trials:
    study.enqueue_trial(params=trial.params)

study.optimize(objective, n_trials=len(test_trials))

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/jupyter-group55/snn-encoder-test/.venv/lib/python3.12/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/jupyter-group55/snn-encoder-test/notebooks/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/jupyter-group55/snn-encoder-test/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/home/jupyter-group55/snn-encoder-test/.venv/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

[I 2025-05-08 22:51:57,308] Trial 25 finished with value: 0.204515278339386 and parameters: {'threshold': 0.06560935122788826, 'slope': 19.515178794292677, 'beta': 0.7485738238159846, 'dropout_rate1': 0.41280634724450427, 'dropout_rate2': 0.7297685730505821, 'lr': 3.9738292608377826e-05, 'weight_decay': 1.95489622971466e-05, 'scheduler_factor': 0.4689936503974681, 'scheduler_patience': 8, 'win_size': 4, 'cutoff': 0.2634325069849827, 'encoder_threshold': 0.03377124060390732}. Best is trial 16 with value: 0.16201859712600708.


In [12]:
logger.info(f"Encoder: BSA Encoding,trial, best_param: {study.best_params}")
logger.info(f"Encoder: BSA Encoding,trial, best_score: {study.best_value}")

2025-05-08 22:51:57.778 | INFO     | __main__:<module>:1 - Encoder: BSA Encoding,trial, best_param: {'threshold': 0.05837571787591716, 'slope': 6.7082922718644795, 'beta': 0.6703615532512789, 'dropout_rate1': 0.3867650737818362, 'dropout_rate2': 0.2650897828875425, 'lr': 6.82947151662786e-05, 'weight_decay': 3.48210294726769e-06, 'scheduler_factor': 0.3434576235217472, 'scheduler_patience': 10, 'win_size': 3, 'cutoff': 0.7497922013805774, 'encoder_threshold': 0.015185576141913593}
2025-05-08 22:51:58.255 | INFO     | __main__:<module>:2 - Encoder: BSA Encoding,trial, best_score: 0.16201859712600708
